In [5]:
import base64 as ii_b6
import datetime as ii_dt
import pandas as ii_pd
import pathlib as ii_pt
import requests as ii_rq
import time as ii_tm

In [7]:
cc_url_hd = 'https://api.github.com/repos/'
cc_own = 'oxxpeh'
cc_repo = 'pub'
cc_tkn = 'API_TKNdd_hd = {
  'Accept': 'application/vnd.github.v3+json' ,
  'Authorization': f"token {cc_tkn}"}
ll_xx =[
['vi','/traffic/views'],
['cl','/traffic/clones'],
['rf','/traffic/popular/referrers'],
['pt','/traffic/popular/paths'],
]
dd_00 = {}

In [ ]:
dt = ii_dt.datetime.now()
for ll_nn in ll_xx:
  dd_00[ll_nn[0]] = ii_rq.get(f"{cc_url_hd}{cc_own}/{cc_repo}{ll_nn[1]}",headers=dd_hd).json()
  if ll_nn[0] == 'vi':
    dd_00[f"df_{ll_nn[0]}"] = ii_pd.json_normalize(dd_00[ll_nn[0]]['views'])
    dd_00[f"df_{ll_nn[0]}"]['timestamp'] = ii_pd.to_datetime(dd_00[f"df_{ll_nn[0]}"]['timestamp'])
  elif ll_nn[0] == 'cl':
    dd_00[f"df_{ll_nn[0]}"] = ii_pd.json_normalize(dd_00[ll_nn[0]]['clones'])
    dd_00[f"df_{ll_nn[0]}"]['timestamp'] = ii_pd.to_datetime(dd_00[f"df_{ll_nn[0]}"]['timestamp'])
  else:
    dd_00[f"df_{ll_nn[0]}"] = ii_pd.json_normalize(dd_00[ll_nn[0]])
    dd_00[f"df_{ll_nn[0]}"]['timestamp'] = dt
    lln = [dd_00[f"df_{ll_nn[0]}"].columns[-1]] + dd_00[f"df_{ll_nn[0]}"].columns[:-1].tolist()
    dd_00[f"df_{ll_nn[0]}"] = dd_00[f"df_{ll_nn[0]}"][lln]
  #dd_00[f"df_{ll_nn[0]}"].to_csv(f"{ll_nn[0]}.csv",index=False)
  ii_tm.sleep(1)

In [ ]:
for ll_nn in ll_xx:
  nn = ll_nn[0]
  df_o = ii_pd.read_csv(f"{nn}.csv")
  df_n = dd_00[f"df_{nn}"]
  if nn in ['vi','cl']:
    dt_o = df_o['timestamp'].max()
    df_oo = df_o[df_o['timestamp'] < dt_o]
    dd_00[f"df_{nn}_n"] = ii_pd.concat([df_oo,df_n])
  else:
    dd_00[f"df_{nn}_n"] = ii_pd.concat([df_o, df_n])
  ii_pt.Path(f"{nn}.csv").rename(ii_pt.Path(f"{nn}_old.csv"))
  dd_00[f"df_{nn}_n"].to_csv(f"{nn}.csv",index=False)

In [ ]:
dd_00

{'vi': {'count': 105,
  'uniques': 4,
  'views': [{'timestamp': '2025-03-16T00:00:00Z', 'count': 7, 'uniques': 1},
   {'timestamp': '2025-03-17T00:00:00Z', 'count': 14, 'uniques': 2},
   {'timestamp': '2025-03-18T00:00:00Z', 'count': 25, 'uniques': 2},
   {'timestamp': '2025-03-19T00:00:00Z', 'count': 11, 'uniques': 2},
   {'timestamp': '2025-03-23T00:00:00Z', 'count': 8, 'uniques': 1},
   {'timestamp': '2025-03-25T00:00:00Z', 'count': 4, 'uniques': 2},
   {'timestamp': '2025-03-26T00:00:00Z', 'count': 6, 'uniques': 2},
   {'timestamp': '2025-03-27T00:00:00Z', 'count': 17, 'uniques': 2},
   {'timestamp': '2025-03-28T00:00:00Z', 'count': 12, 'uniques': 1},
   {'timestamp': '2025-03-29T00:00:00Z', 'count': 1, 'uniques': 1}]},
 'cl': {'count': 17,
  'uniques': 11,
  'clones': [{'timestamp': '2025-03-17T00:00:00Z', 'count': 5, 'uniques': 4},
   {'timestamp': '2025-03-19T00:00:00Z', 'count': 6, 'uniques': 4},
   {'timestamp': '2025-03-22T00:00:00Z', 'count': 1, 'uniques': 1},
   {'timestamp

In [46]:
def ff_get_sha():
  url = f"{cc_url_hd}{cc_own}/{cc_repo}/git/refs/heads/main"  # mainブランチを使用

  # ヘッダーに認証情報を追加
  ## headers = {
  ##     'Authorization': f'token {cc_tkn}',
  ##     'Accept': 'application/vnd.github.v3+json'
  ## }

  # 最新のコミットを取得
  rt = ii_rq.get(url, headers=dd_hd)
  if rt.status_code == 200:
    sha_tl = rt.json()['object']['sha']
    print(f"最新のコミットSHA: {sha_tl}")
  else:
    print(f"エラーが発生しました: {rt.status_code} {rt.text}")
  return(sha_tl)

In [55]:
def ff_cm(pt_src, aas_cm, pt_dst):
   """
   """
   sha_tl = ff_get_sha()

   with open(pt_src, 'r') as fh_r:
       oo_src = fh_r.read()

   oo_ec_src = ii_b6.b64encode(oo_src.encode()).decode()

   # 新しいブロブを作成
   url_bl = f"{cc_url_hd}{cc_own}/{cc_repo}/git/blobs"
   oo_bl = {
       'content': oo_ec_src,
       'encoding': 'base64'
   }
   rt_bl = ii_rq.post(url_bl, json=oo_bl, headers=dd_hd)
   ii_tm.sleep(1)
   if rt_bl.status_code == 201:
       sha_bl = rt_bl.json()['sha']
       print(f"新しいブロブSHA: {sha_bl}")

       # 新しいツリーを作成
       url_tr = f'{cc_url_hd}{cc_own}/{cc_repo}/git/trees'
       dd_tr = {
           'base_tree': sha_tl,
           'tree': [
               {
                   'path': pt_dst,  # 変更するファイルのパス
                   'mode': '100644',  # ファイルモード
                   'type': 'blob',
                   'sha': sha_bl
               }
           ]
       }
       rt_tr = ii_rq.post(url_tr, json=dd_tr, headers=dd_hd)
       ii_tm.sleep(1)
       if rt_tr.status_code == 201:
           sha_tr = rt_tr.json()['sha']
           print(f"新しいツリーSHA: {sha_tr}")

           # 新しいコミットを作成
           url_cm = f"{cc_url_hd}{cc_own}/{cc_repo}/git/commits"
           dd_cm = {
               'message': aas_cm,
               'tree': sha_tr,
               'parents': [sha_tl]
           }
           rt_cm = ii_rq.post(url_cm, json=dd_cm, headers=dd_hd)
           ii_tm.sleep(1)
           if rt_cm.status_code == 201:
               sha_new = rt_cm.json()['sha']
               print(f"新しいコミットSHA: {sha_new}")

               # ブランチを更新
               url_ref = f"{cc_url_hd}{cc_own}/{cc_repo}/git/refs/heads/main"
               dd_ref = {
                   'sha': sha_new
               }
               rt_ref = ii_rq.patch(url_ref, json=dd_ref, headers=dd_hd)

               if rt_ref.status_code == 200:
                   print(f"ブランチが更新されました。 {sha_new}")
               else:
                   print(f"ブランチの更新に失敗しました: {rt_ref.status_code} {rt_ref.text}")
           else:
               print(f"コミットの作成に失敗しました: {rt_cm.status_code}")


残骸

In [52]:
sha_tl =ff_get_sha()

最新のコミットSHA: 36975522115a3c62544726cd55d89355976a117b


In [51]:
# 変更するファイルの情報
pt_src = 'rf.csv'  # 変更するファイルのパス
aas_cm = 'API Update file.txt'  # コミットメッセージ

# 変更内容を取得
with open(pt_src, 'r') as fh_r:
    oo_src = fh_r.read()

# Base64エンコード
# import base64 as ii_b6
oo_ec_src = ii_b6.b64encode(oo_src.encode()).decode()

# 新しいブロブを作成
url_bl = f"{cc_url_hd}{cc_own}/{cc_repo}/git/blobs"
oo_bl = {
    'content': oo_ec_src,
    'encoding': 'base64'
}
rt_bl = ii_rq.post(url_bl, json=oo_bl, headers=dd_hd)

if rt_bl.status_code == 201:
    sha_bl = rt_bl.json()['sha']
    print(f"新しいブロブSHA: {sha_bl}")

    # 新しいツリーを作成
    url_tr = f'{cc_url_hd}{cc_own}/{cc_repo}/git/trees'
    dd_tr = {
        'base_tree': sha_tl,
        'tree': [
            {
                'path': 'rf.csv',  # 変更するファイルのパス
                'mode': '100644',  # ファイルモード
                'type': 'blob',
                'sha': sha_bl
            }
        ]
    }
    rt_tr = ii_rq.post(url_tr, json=dd_tr, headers=dd_hd)

    if rt_tr.status_code == 201:
        sha_tr = rt_tr.json()['sha']
        print(f"新しいツリーSHA: {sha_tr}")

        # 新しいコミットを作成
        url_cm = f"{cc_url_hd}{cc_own}/{cc_repo}/git/commits"
        dd_cm = {
            'message': aas_cm,
            'tree': sha_tr,
            'parents': [sha_tl]
        }
        rt_cm = ii_rq.post(url_cm, json=dd_cm, headers=dd_hd)

        if rt_cm.status_code == 201:
            sha_new = rt_cm.json()['sha']
            print(f"新しいコミットSHA: {sha_new}")

            # ブランチを更新
            url_ref = f"{cc_url_hd}{cc_own}/{cc_repo}/git/refs/heads/main"
            dd_ref = {
                'sha': sha_new
            }
            ret_ref = ii_rq.patch(url_ref, json=dd_ref, headers=dd_hd)

            if ret_ref.status_code == 200:
                print(f"ブランチが更新されました。 {sha_new}")
            else:
                print(f"ブランチの更新に失敗しました: {rt_ref.status_code} {rt_ref.text}")
        else:
            print(f"コミットの作成に失敗しました: {rt_cm.status_code}")




新しいブロブSHA: f87c1c975bafe0a12e6e0ade43ffb454a2a56e4d
新しいツリーSHA: eeb2403bd8f1889aac684a23cc0ba44c98a7a7e4
新しいコミットSHA: 36975522115a3c62544726cd55d89355976a117b
ブランチが更新されました。 36975522115a3c62544726cd55d89355976a117b


In [24]:
ret_ref.text

'{"ref":"refs/heads/main","node_id":"REF_kwDOKV5-hq9yZWZzL2hlYWRzL21haW4","url":"https://api.github.com/repos/oxxpeh/pub/git/refs/heads/main","object":{"sha":"22a881f8b4457c834e58d8ad6a8f7df48f0b217f","type":"commit","url":"https://api.github.com/repos/oxxpeh/pub/git/commits/22a881f8b4457c834e58d8ad6a8f7df48f0b217f"}}'

In [48]:
dd_ref = {'sha': sha_tl}

In [49]:
rt_ref = ii_rq.patch(url_ref, json=dd_ref, headers=dd_hd)
print(rt_ref.text)

{"ref":"refs/heads/main","node_id":"REF_kwDOKV5-hq9yZWZzL2hlYWRzL21haW4","url":"https://api.github.com/repos/oxxpeh/pub/git/refs/heads/main","object":{"sha":"90056037f337d180e34303047256a63a335f348e","type":"commit","url":"https://api.github.com/repos/oxxpeh/pub/git/commits/90056037f337d180e34303047256a63a335f348e"}}


In [38]:
rt_ref.text

'{"ref":"refs/heads/main","node_id":"REF_kwDOKV5-hq9yZWZzL2hlYWRzL21haW4","url":"https://api.github.com/repos/oxxpeh/pub/git/refs/heads/main","object":{"sha":"90056037f337d180e34303047256a63a335f348e","type":"commit","url":"https://api.github.com/repos/oxxpeh/pub/git/commits/90056037f337d180e34303047256a63a335f348e"}}'

In [56]:
ff_cm('pt.csv','API Update pt.csv','python/github_api/pt.csv')

最新のコミットSHA: 7ccfd13e9d375eea7c7bc0e0fe42a5cb8ee46907
新しいブロブSHA: 21202f2153292828d8945a03ca4f02d3f31e51e9
新しいツリーSHA: 9cc9c3de37a32f524e5598b4a0604a97e33f5a1f
新しいコミットSHA: ba0ce86a50583dce28a8f8bfe3a05c348e3881f9
ブランチが更新されました。 ba0ce86a50583dce28a8f8bfe3a05c348e3881f9


In [57]:
ff_cm('vi.csv','API Update vi.csv','python/github_api/vi.csv')
ff_cm('cl.csv','API Update cl.csv','python/github_api/cl.csv')

最新のコミットSHA: ba0ce86a50583dce28a8f8bfe3a05c348e3881f9
新しいブロブSHA: 4132051628a36db0d6769fdedb24e6375eb5d409
新しいツリーSHA: 73c47685f50ca0bfe8187cd69bf14fb4a4c835ae
新しいコミットSHA: 44b45da3053c0f51a27ee803b04fdcf08cabafcf
ブランチが更新されました。 44b45da3053c0f51a27ee803b04fdcf08cabafcf
最新のコミットSHA: 44b45da3053c0f51a27ee803b04fdcf08cabafcf
新しいブロブSHA: 21b27792f65f9ed9c924c65ab69a05e3684531f6
新しいツリーSHA: 0ced0a321d033ad1b1deb6b6ad8b29f767813019
新しいコミットSHA: 6e9d479e274cd4e09475819ce14c2874a7bbc8a8
ブランチが更新されました。 6e9d479e274cd4e09475819ce14c2874a7bbc8a8


In [ ]:
rt.text

'[{"referrer":"github.com","count":31,"uniques":3},{"referrer":"Bing","count":15,"uniques":3},{"referrer":"oxxpeh.github.io","count":4,"uniques":1},{"referrer":"DuckDuckGo","count":1,"uniques":1}]'

In [ ]:
dd_n = dd_00.copy()

In [ ]:
dd_n

{'vi': {'count': 5,
  'uniques': 2,
  'views': [{'timestamp': '2025-03-30T00:00:00Z', 'count': 3, 'uniques': 1},
   {'timestamp': '2025-03-31T00:00:00Z', 'count': 1, 'uniques': 1},
   {'timestamp': '2025-04-05T00:00:00Z', 'count': 1, 'uniques': 1}]},
 'df_vi':                   timestamp  count  uniques
 0 2025-03-30 00:00:00+00:00      3        1
 1 2025-03-31 00:00:00+00:00      1        1
 2 2025-04-05 00:00:00+00:00      1        1,
 'cl': {'count': 8,
  'uniques': 8,
  'clones': [{'timestamp': '2025-03-29T00:00:00Z', 'count': 1, 'uniques': 1},
   {'timestamp': '2025-03-31T00:00:00Z', 'count': 3, 'uniques': 3},
   {'timestamp': '2025-04-01T00:00:00Z', 'count': 1, 'uniques': 1},
   {'timestamp': '2025-04-05T00:00:00Z', 'count': 1, 'uniques': 1},
   {'timestamp': '2025-04-10T00:00:00Z', 'count': 1, 'uniques': 1},
   {'timestamp': '2025-04-11T00:00:00Z', 'count': 1, 'uniques': 1}]},
 'df_cl':                   timestamp  count  uniques
 0 2025-03-29 00:00:00+00:00      1        1
 1 2

In [ ]:
f"{cc_url_hd}{cc_own}/{cc_repo}/traffic/popular/referrers"

'https://api.github.com/repos/oxxpeh/pub/traffic/popular/referrers'

In [ ]:
dd_00[f"df_{ll_nn[0]}"]

,path,title,count,uniques,date
0,/oxxpeh/pub,oxxpeh/pub: docker incus とかいろいろな物置,27,4,2025-03-30 06:06:26.018333
1,/oxxpeh/pub/blob/main/python/10s-chart/README.md,pub/python/10s-chart/README.md at main · oxxpe...,20,4,2025-03-30 06:06:26.018333
2,/oxxpeh/pub/tree/main/python/10s-chart,pub/python/10s-chart at main · oxxpeh/pub · Gi...,8,3,2025-03-30 06:06:26.018333
3,/oxxpeh/pub/pulse,Pulse · oxxpeh/pub · GitHub,5,2,2025-03-30 06:06:26.018333
4,/oxxpeh/pub/deployments,Deployments · oxxpeh/pub,4,1,2025-03-30 06:06:26.018333
5,/oxxpeh/pub/graphs/traffic,Traffic · oxxpeh/pub,4,1,2025-03-30 06:06:26.018333
6,/oxxpeh/pub/blob/main/python/10s-chart/pys_ch.py,pub/python/10s-chart/pys_ch.py at main · oxxpe...,3,2,2025-03-30 06:06:26.018333
7,/oxxpeh/pub/network,Network Graph · oxxpeh/pub · GitHub,2,2,2025-03-30 06:06:26.018333
8,/oxxpeh/pub/edit/main/python/10s-chart/README.md,Editing pub/python/10s-chart/README.md at main...,2,1,2025-03-30 06:06:26.018333
9,/oxxpeh/pub/tree/main/suji,pub/suji at main · oxxpeh/pub,2,1,2025-03-30 06:06:26.018333


In [ ]:
print(type(dd_00['vi']))
print(type(dd_00['vi']['views']))
print(type(dd_00['rf']))

<class 'dict'>
<class 'list'>
<class 'list'>


In [ ]:
dd_00[ll_nn[0]]

[{'referrer': 'github.com', 'count': 26, 'uniques': 3},
 {'referrer': 'Bing', 'count': 15, 'uniques': 3},
 {'referrer': 'oxxpeh.github.io', 'count': 2, 'uniques': 1},
 {'referrer': 'DuckDuckGo', 'count': 1, 'uniques': 1}]

In [ ]:
df_vi0 = ii_pd.read_json('vi.json')
df_cl0 = ii_pd.read_json('cl.json')
df_pt = ii_pd.read_json('pat.json')
df_rf = ii_pd.read_json('ref.json')

In [ ]:
df_vi = ii_pd.json_normalize(df_vi0['views'])
df_cl = ii_pd.json_normalize(df_cl0['clones'])

In [ ]:
df_vi['timestamp'] = ii_pd.to_datetime(df_vi['timestamp'])
df_cl['timestamp'] = ii_pd.to_datetime(df_cl['timestamp'])

In [ ]:
df_vi

,timestamp,count,uniques
0,2025-03-15 00:00:00+00:00,69,2
1,2025-03-16 00:00:00+00:00,23,2
2,2025-03-17 00:00:00+00:00,14,2
3,2025-03-18 00:00:00+00:00,25,2
4,2025-03-19 00:00:00+00:00,11,2
5,2025-03-23 00:00:00+00:00,8,1
6,2025-03-25 00:00:00+00:00,4,2
7,2025-03-26 00:00:00+00:00,6,2
8,2025-03-27 00:00:00+00:00,17,2


In [ ]:
ii_dt.datetime.strptime('2025-03-27','%Y-%m-%d')

datetime.datetime(2025, 3, 27, 0, 0)

In [ ]:
ii_pt.Path('vi.csv').rename(ii_pt.Path('vi_x.csv'))

PosixPath('vi_x.csv')

In [ ]:
df_rf['timestamp'] = ii_dt.datetime.strptime('2025-03-27','%Y-%m-%d')
df_pt['timestamp'] = ii_dt.datetime.strptime('2025-03-27','%Y-%m-%d')

In [ ]:
df_rf

,referrer,count,uniques,timestamp
0,github.com,46,3,2025-03-27
1,Bing,15,3,2025-03-27
2,DuckDuckGo,5,2,2025-03-27
3,oxxpeh.github.io,4,1,2025-03-27


In [ ]:
dd_n['df_cl']

,timestamp,count,uniques
0,2025-03-29 00:00:00+00:00,1,1
1,2025-03-31 00:00:00+00:00,3,3
2,2025-04-01 00:00:00+00:00,1,1
3,2025-04-05 00:00:00+00:00,1,1
4,2025-04-10 00:00:00+00:00,1,1
5,2025-04-11 00:00:00+00:00,1,1


In [ ]:
df_cl

,timestamp,count,uniques
0,2025-03-15 00:00:00+00:00,32,11
1,2025-03-16 00:00:00+00:00,13,7
2,2025-03-17 00:00:00+00:00,5,4
3,2025-03-19 00:00:00+00:00,6,4
4,2025-03-22 00:00:00+00:00,1,1


In [ ]:
df_vi

,timestamp,count,uniques
0,2025-03-15 00:00:00+00:00,69,2
1,2025-03-16 00:00:00+00:00,23,2
2,2025-03-17 00:00:00+00:00,14,2
3,2025-03-18 00:00:00+00:00,25,2
4,2025-03-19 00:00:00+00:00,11,2
5,2025-03-23 00:00:00+00:00,8,1
6,2025-03-25 00:00:00+00:00,4,2
7,2025-03-26 00:00:00+00:00,6,2
8,2025-03-27 00:00:00+00:00,17,2


In [ ]:
dd_n['df_vi']

,timestamp,count,uniques
0,2025-03-30 00:00:00+00:00,3,1
1,2025-03-31 00:00:00+00:00,1,1
2,2025-04-05 00:00:00+00:00,1,1


In [ ]:
dd_n['df_rf']

,referrer,count,uniques,timestamp
0,DuckDuckGo,2,1,2025-04-12 08:45:45.021855


In [ ]:
dd_00[f"df_{ll_nn[0]}"]['date'] = ii_dt.datetime.now()

In [ ]:
df_vi0

,count,uniques,views
0,177,4,"{'timestamp': '2025-03-15T00:00:00Z', 'count':..."
1,177,4,"{'timestamp': '2025-03-16T00:00:00Z', 'count':..."
2,177,4,"{'timestamp': '2025-03-17T00:00:00Z', 'count':..."
3,177,4,"{'timestamp': '2025-03-18T00:00:00Z', 'count':..."
4,177,4,"{'timestamp': '2025-03-19T00:00:00Z', 'count':..."
5,177,4,"{'timestamp': '2025-03-23T00:00:00Z', 'count':..."
6,177,4,"{'timestamp': '2025-03-25T00:00:00Z', 'count':..."
7,177,4,"{'timestamp': '2025-03-26T00:00:00Z', 'count':..."
8,177,4,"{'timestamp': '2025-03-27T00:00:00Z', 'count':..."


In [ ]:
df_cl0

,count,uniques,clones
0,57,17,"{'timestamp': '2025-03-15T00:00:00Z', 'count':..."
1,57,17,"{'timestamp': '2025-03-16T00:00:00Z', 'count':..."
2,57,17,"{'timestamp': '2025-03-17T00:00:00Z', 'count':..."
3,57,17,"{'timestamp': '2025-03-19T00:00:00Z', 'count':..."
4,57,17,"{'timestamp': '2025-03-22T00:00:00Z', 'count':..."


In [ ]:
df_vi0['views']

,views
0,"{'timestamp': '2025-03-15T00:00:00Z', 'count':..."
1,"{'timestamp': '2025-03-16T00:00:00Z', 'count':..."
2,"{'timestamp': '2025-03-17T00:00:00Z', 'count':..."
3,"{'timestamp': '2025-03-18T00:00:00Z', 'count':..."
4,"{'timestamp': '2025-03-19T00:00:00Z', 'count':..."
5,"{'timestamp': '2025-03-23T00:00:00Z', 'count':..."
6,"{'timestamp': '2025-03-25T00:00:00Z', 'count':..."
7,"{'timestamp': '2025-03-26T00:00:00Z', 'count':..."
8,"{'timestamp': '2025-03-27T00:00:00Z', 'count':..."


In [ ]:
df_vi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  9 non-null      object
 1   count      9 non-null      int64 
 2   uniques    9 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 348.0+ bytes


In [ ]:
rt= ii_rq.get(f"{cc_url_hd}{cc_own}/{cc_repo}/traffic/popular/referrers",headers=dd_hd)

In [2]:
df_op= ii_pd.read_csv(f"pt.csv")
df_or= ii_pd.read_csv(f"rf.csv")

In [3]:
llp = [df_op.columns[-1]] + df_op.columns[:-1].tolist()
df_np = df_op[llp]
llr = [df_or.columns[-1]] + df_or.columns[:-1].tolist()
df_nr = df_or[llr]
print(df_nr)
print(df_np)

                    timestamp          referrer  count  uniques
0  2025-03-27 00:00:00.000000        github.com     46        3
1  2025-03-27 00:00:00.000000              Bing     15        3
2  2025-03-27 00:00:00.000000        DuckDuckGo      5        2
3  2025-03-27 00:00:00.000000  oxxpeh.github.io      4        1
4  2025-04-12 08:45:45.021855        DuckDuckGo      2        1
                     timestamp  \
0   2025-03-27 00:00:00.000000   
1   2025-03-27 00:00:00.000000   
2   2025-03-27 00:00:00.000000   
3   2025-03-27 00:00:00.000000   
4   2025-03-27 00:00:00.000000   
5   2025-03-27 00:00:00.000000   
6   2025-03-27 00:00:00.000000   
7   2025-03-27 00:00:00.000000   
8   2025-03-27 00:00:00.000000   
9   2025-03-27 00:00:00.000000   
10  2025-04-12 08:45:45.021855   
11  2025-04-12 08:45:45.021855   

                                                 path  \
0    /oxxpeh/pub/blob/main/python/10s-chart/README.md   
1                                         /oxxpeh/pub   
2 

In [4]:
df_nr.to_csv(f"rf.csv",index=False)
df_np.to_csv(f"pt.csv",index=False)

In [ ]:
df_vi['timestamp'].astype('datetime64[ns]')

ValueError: cannot supply both a tz and a timezone-naive dtype (i.e. datetime64[ns])

In [ ]:
ii_pd.to_datetime(df_vi['timestamp'])

,timestamp
0,2025-03-15 00:00:00+00:00
1,2025-03-16 00:00:00+00:00
2,2025-03-17 00:00:00+00:00
3,2025-03-18 00:00:00+00:00
4,2025-03-19 00:00:00+00:00
5,2025-03-23 00:00:00+00:00
6,2025-03-25 00:00:00+00:00
7,2025-03-26 00:00:00+00:00
8,2025-03-27 00:00:00+00:00


In [ ]:
df_cl

,timestamp,count,uniques
0,2025-03-15T00:00:00Z,32,11
1,2025-03-16T00:00:00Z,13,7
2,2025-03-17T00:00:00Z,5,4
3,2025-03-19T00:00:00Z,6,4
4,2025-03-22T00:00:00Z,1,1


In [ ]:
df_rf

,referrer,count,uniques
0,github.com,46,3
1,Bing,15,3
2,DuckDuckGo,5,2
3,oxxpeh.github.io,4,1


In [ ]:
dd_o = {}
dd_xx = {}
dd_o['vi'] = df_vi
dd_o['cl'] = df_cl
dd_o['rf'] = df_rf
dd_o['pt'] = df_pt
ll = ['vi','cl','rf','pt']
for nn in ll:
  if nn in ['vi','cl ']:
    dt_o = dd_o[nn]['timestamp'].max()
    df_oo = dd_o[nn][dd_o[nn]['timestamp'] < dt_o]
    dd_xx[f"df_x_{nn}"] = ii_pd.concat([df_oo,dd_n[f"df_{nn}"]])
  else:
    dd_xx[f"df_x_{nn}"] = ii_pd.concat([dd_o[nn],dd_n[f"df_{nn}"]])

In [ ]:
for nn in ll:
  dd_o[f"{nn}"].to_csv(f"{nn}.csv",index=False)

In [ ]:
dd_xx

{'df_x_vi':                   timestamp  count  uniques
 0 2025-03-15 00:00:00+00:00     69        2
 1 2025-03-16 00:00:00+00:00     23        2
 2 2025-03-17 00:00:00+00:00     14        2
 3 2025-03-18 00:00:00+00:00     25        2
 4 2025-03-19 00:00:00+00:00     11        2
 5 2025-03-23 00:00:00+00:00      8        1
 6 2025-03-25 00:00:00+00:00      4        2
 7 2025-03-26 00:00:00+00:00      6        2
 0 2025-03-30 00:00:00+00:00      3        1
 1 2025-03-31 00:00:00+00:00      1        1
 2 2025-04-05 00:00:00+00:00      1        1,
 'df_x_cl':                   timestamp  count  uniques
 0 2025-03-15 00:00:00+00:00     32       11
 1 2025-03-16 00:00:00+00:00     13        7
 2 2025-03-17 00:00:00+00:00      5        4
 3 2025-03-19 00:00:00+00:00      6        4
 4 2025-03-22 00:00:00+00:00      1        1
 0 2025-03-29 00:00:00+00:00      1        1
 1 2025-03-31 00:00:00+00:00      3        3
 2 2025-04-01 00:00:00+00:00      1        1
 3 2025-04-05 00:00:00+00:00    

In [ ]:
ii_pd.json_normalize(df_vi0.iloc[:,2])

,timestamp,count,uniques
0,2025-03-15T00:00:00Z,69,2
1,2025-03-16T00:00:00Z,23,2
2,2025-03-17T00:00:00Z,14,2
3,2025-03-18T00:00:00Z,25,2
4,2025-03-19T00:00:00Z,11,2
5,2025-03-23T00:00:00Z,8,1
6,2025-03-25T00:00:00Z,4,2
7,2025-03-26T00:00:00Z,6,2
8,2025-03-27T00:00:00Z,17,2
